In [196]:
import torch.nn.functional as F
from torch.autograd import Variable
import copy
from collections import Counter
import torch 
import numpy as np
import torch.nn as nn

In [197]:
def read_data(path):
    f = open(path, "r")
    lines = f.readlines()
    f.close()
    data = []
    for l in lines:
        labelSplit = l.replace('\n','').split(' ', 1)
        data.append([labelSplit[0], [word.lower() for word in labelSplit[1].split()]])
    return data 

data = read_data('./questions.txt')

In [198]:
def remove_stop_words(data, path):
    stop_words = []
    with open('stop_words.txt') as f:
        stop_words = [word for line in f for word in line.split(",")]
    data_without_stop_words = []
    for k, v in data:
        words = [t for t in v if t not in stop_words]
        data_without_stop_words.append((k, words))
    return data_without_stop_words

data = remove_stop_words(data, './stop_words.txt')

In [199]:
def get_labels(data):
    _labels = []
    for k,v in data:
        _labels.append(k)   
    _unique_label = list(set(_labels))
    _unique_label_dict = {}
    for k,v in enumerate(_unique_label):
        _unique_label_dict[v] = k
    return _unique_label_dict

labels = get_labels(data)

In [200]:
def encode_labels(data, labels):
    encoded_labels = {}
    for i in range(0, 50): 
        label = [0] * 50
        label[i] = 1
        encoded_labels[i] = label
        
    cleaned_data = []
    for k,v in data:
        cleaned_data.append((encoded_labels[labels[k]],v))
        
    return cleaned_data

data = encode_labels(data, labels)

In [201]:
def create_indexed_vocab(data):
    vocab = []
    for _, sent in data:
        for word in sent:
            vocab.append(word)
    count = Counter(vocab)
    count = {w : count[w] for w in count if count[w] >= 2}
    vocab = []
    for k, v in count.items():
        vocab.append(k)
    indexed_vocab = {word: idx for idx, word in enumerate(vocab)}
    return indexed_vocab

indexed_vocab = create_indexed_vocab(data)
indexed_vocab['#UNK#'] = len(indexed_vocab)

In [202]:
def load_glove_embeddings(path, indexed_vocab, embedding_dim=300):
    with open(path) as f:
        embeddings = np.zeros((len(indexed_vocab), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = indexed_vocab.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
            else:
                vector = np.array(values[1:], dtype='float32')
                embeddings[len(indexed_vocab) - 1] = vector
        return torch.from_numpy(embeddings).float()

glove = load_glove_embeddings('./glove.small.txt', indexed_vocab)
embeddings_random = nn.Embedding(glove.size(0), glove.size(1))
embeddings_pretrained = nn.Embedding.from_pretrained(glove, freeze=False)

In [203]:
def split_train_test(data, test_ratio):
    data_copy = copy.deepcopy(data)
    np.random.shuffle(data_copy)
    test_set_size = int(len(data) * test_ratio)
    test = data_copy[:test_set_size]
    train = data_copy[test_set_size:]
    return train, test

train, test = split_train_test(data, 0.1)

In [204]:
class BOWClassifier(nn.Module):
    def __init__(self,input_size,hidden_size, num_labels):
        super(BOWClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size,num_labels)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return torch.LongTensor(output)

In [205]:
input_size = 300
hidden_size = 10
num_labels = 50
bow = BOWClassifier(input_size, hidden_size, num_labels)

In [213]:
def make_bow_vector(sentence, indexed_vocab):
    vec = [0] * 300
    pt_tensor= torch.FloatTensor(vec)
    count = 0
    for word in sentence:
        if word not in indexed_vocab:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab['#UNK#']]))[0])
        else:
            pt_tensor = torch.add(pt_tensor, embeddings_pretrained(torch.LongTensor([indexed_vocab[word]]))[0])
    return pt_tensor

In [214]:
loss_function = nn.MSELoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [216]:

epoch = 2
for epoch in range(epoch):
    for l, d in data:
        opt.zero_grad()
        bow_vec = make_bow_vector(d, indexed_vocab) 
        y_pred = bow(bow_vec)
        loss = loss_function(y_pred, l)
        print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
        loss.backward()
        opt.step()

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2' in call to _th_mm